In [1]:
!pip install pytesseract opencv-python numpy

In [2]:
!sudo apt install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,760 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 24.7 MB/s eta 0:00:00


In [4]:
!pip install transformers sentencepiece


In [5]:
!wget https://github.com/tesseract-ocr/tessdata_best/raw/main/eng.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/
!wget https://github.com/tesseract-ocr/tessdata_fast/raw/main/eng.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/

--2025-02-13 11:39:35--  https://github.com/tesseract-ocr/tessdata_best/raw/main/eng.traineddata
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/eng.traineddata [following]
--2025-02-13 11:39:36--  https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/eng.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15400601 (15M) [application/octet-stream]
Saving to: ‘/usr/share/tesseract-ocr/4.00/tessdata/eng.traineddata.1’

eng.traineddata.1   100%[===================>]  14.69M  --.-KB/s    in 0.1s    

2025-02-13 11:39:36 (127 MB/s) - ‘/usr/share/t

In [6]:
import cv2
import pytesseract
import numpy as np

from ultralytics import YOLO
import torch


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
# Load trained YOLO model
model = YOLO("/content/drive/MyDrive/yolov8_trained.pt")

In [8]:
# Function to detect bubbles in an image
def detect_bubbles(image_path, conf_threshold=0.3):
    # Load image
    img = cv2.imread(image_path)

    # Perform detection
    results = model(img)

    # Extract bounding boxes
    bubbles = []
    for r in results:
        for box in r.boxes:
          if box.conf[0] >= conf_threshold:  # Filter out low-confidence detections
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            bubbles.append((x1, y1, x2, y2))

    return img, bubbles



# Function to extract text from detected bubbles
def extract_text_from_bubbles(img, bubbles, tessdata_dir="/usr/share/tesseract-ocr/4.00/tessdata", model_type="fast"):
    # Detect bubbles
    #img, bubbles = detect_bubbles(image_path)

    extracted_texts = []

    for (x1, y1, x2, y2) in bubbles:
        # Crop the detected text bubble
        cropped_bubble = img[y1:y2, x1:x2]

        # Convert to grayscale
        gray = cv2.cvtColor(cropped_bubble, cv2.COLOR_BGR2GRAY)

        _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Use Tesseract OCR with improved parameters
        tess_model = "eng"
        if model_type == "best":
            tess_model = f"{tessdata_dir}/eng_best"
        elif model_type == "fast":
            tess_model = f"{tessdata_dir}/eng_fast"

        # Apply OCR
        text = pytesseract.image_to_string(gray, lang="eng")
        custom_oem_psm_config = f'--tessdata-dir {tessdata_dir} --psm 6'
        #text = pytesseract.image_to_string(binary, lang=tess_model, config="--oem 1 --psm 6")

        extracted_texts.append(text)

        # Draw bounding box and text (for visualization)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        #cv2.putText(img, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the image with detected text
    cv2.imwrite("detected_bubbles.jpg", img)

    return extracted_texts



In [ ]:
_, bubbles = detect_bubbles('data_02.jpg')

bubbles


0: 640x640 7 Manga-pagess, 256.9ms
Speed: 6.1ms preprocess, 256.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


[(173, 413, 429, 516),
 (39, 187, 205, 286),
 (25, 407, 172, 532),
 (35, 315, 203, 415),
 (508, 415, 632, 514),
 (456, 31, 577, 131),
 (82, 38, 200, 143)]

In [ ]:
# !------------------- Text Translation -----------------!

In [9]:
# Load the English-to-French Translation Model
from transformers import MarianMTModel, MarianTokenizer


# Load English-to-French translation model & tokenizer
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model_transformer = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
# Function to translate english text into french
def translate_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Generate translation
    translated_tokens = model_transformer.generate(**inputs)

    # Decode and return translated text
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text


In [ ]:
# !------------------ Replacing previous text with translated text ----------------!

In [24]:
def remove_old_text(img, bubbles):
    """ Clears old text by filling the bubble area with a white background. """
    for (x1, y1, x2, y2) in bubbles:
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), -1)  # Fill with white
    return img

In [28]:
def wrap_text(text, max_width, font, font_scale, thickness):
    """Wraps text to fit within max_width of a bubble."""
    words = text.split(' ')
    wrapped_lines = []
    current_line = ""

    for word in words:
        test_line = current_line + " " + word if current_line else word
        text_size = cv2.getTextSize(test_line, font, font_scale, thickness)[0]
        if text_size[0] <= max_width:
            current_line = test_line
        else:
            wrapped_lines.append(current_line)
            current_line = word

    if current_line:
        wrapped_lines.append(current_line)

    return wrapped_lines

def add_new_text(img, bubbles, translated_texts):
    for (i, (x1, y1, x2, y2)) in enumerate(bubbles):
        text = translated_texts[i]
        font = cv2.FONT_HERSHEY_SIMPLEX
        thickness = 1
        text_color = (0, 0, 0)  # Black color
        font_scale = 0.4  # Fixed font scale

        max_width = x2 - x1 - 10  # Allow some padding
        wrapped_lines = wrap_text(text, max_width, font, font_scale, thickness)

        line_height = cv2.getTextSize("Test", font, font_scale, thickness)[0][1] + 5
        total_text_height = line_height * len(wrapped_lines)

        start_y = y1 + (y2 - y1 - total_text_height) // 2  # Center vertically
        for line in wrapped_lines:
            text_size = cv2.getTextSize(line, font, font_scale, thickness)[0]
            text_x = x1 + (x2 - x1 - text_size[0]) // 2  # Center horizontally
            cv2.putText(img, line, (text_x, start_y), font, font_scale, text_color, thickness, cv2.LINE_AA)
            start_y += line_height

    return img

In [29]:
# Load and process image
image_path = "data_02.jpg"
img, bubbles = detect_bubbles(image_path)
texts = extract_text_from_bubbles(img, bubbles, model_type="best")
translated_texts = [translate_text(text) for text in texts]


# Remove old text and add new translated text
removed_old_text_img = remove_old_text(img, bubbles)
new_img = add_new_text(removed_old_text_img, bubbles, translated_texts)


# Save and display the modified image
output_path = "translated_image.jpg"
cv2.imwrite(output_path, new_img)

#cv2.imshow("Translated Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 640x640 7 Manga-pagess, 239.8ms
Speed: 5.1ms preprocess, 239.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
